# Sentiment Analysis using 1D Convolutional Neural Networks in Keras

In [1]:
# https://learnremote.medium.com/sentiment-analysis-using-1d-convolutional-neural-networks-part-1-f8b6316489a2
# https://github.com/gizenmtl/IMDB-Sentiment-Analysis-and-Text-Classification/tree/master/aclImdb/train

In [13]:
import os
import pickle as pk

imdb_dir = 'D:/SentimentAnalysis/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')
labels = list()
texts = list()


In [14]:
# Processing the labels of the raw IMDB data
for label_type in ['neg', 'pos', 'unsup']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding="utf8")
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            if label_type == 'pos':
                labels.append(1)                
            else:
                labels.append(2)


In [18]:
# Tokenizing the data
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.optimizers import RMSprop

# cut off reviews after 500 words
max_len = 500 
# train on 10000 samples
training_samples = 10000
 # validate on 10000 samples 
validation_samples = 10000
# consider only the top 10000 words
max_words = 10000 

tokenizer_path = 'tokenizer'
# import tokenizer with the consideration for only the top 500 words
tokenizer = Tokenizer(num_words=max_words) 
# fit the tokenizer on the texts
tokenizer.fit_on_texts(texts) 
# convert the texts to sequences
sequences = tokenizer.texts_to_sequences(texts) 

# save the tokenizer
with open(os.path.join(tokenizer_path, 'tokenizer_m1.pickle'), 'wb') as handle:
    pk.dump(tokenizer, handle, protocol=pk.HIGHEST_PROTOCOL)

word_index = tokenizer.word_index
print('Found %s unique tokens. ' % len(word_index))

 # pad the sequence to the required length to ensure uniformity
data = pad_sequences(sequences, maxlen=max_len)
print('Data Shape: {}'.format(data.shape))

labels = np.asarray(labels)
print("Shape of data tensor: ", data.shape)
print("Shape of label tensor: ", labels.shape)

# split the data into training and validation set but before that shuffle it first
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples:training_samples + validation_samples]
y_val = labels[training_samples:training_samples + validation_samples]

# test_data
x_test = data[training_samples+validation_samples:]
y_test = labels[training_samples+validation_samples:]



Found 693 unique tokens. 
Data Shape: (9, 500)
Shape of data tensor:  (9, 500)
Shape of label tensor:  (12,)


In [19]:
# model definition
import keras
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras import Model, layers
from keras import Input

callback_list = [
    keras.callbacks.EarlyStopping(
        patience=1,
        monitor='acc',
    ),
    
    keras.callbacks.TensorBoard(
        log_dir='log_dir_m1',
        histogram_freq=1,
        embeddings_freq=1,
    ),

    keras.callbacks.ModelCheckpoint(
        monitor='val_loss',
        save_best_only=True,
        filepath='model/movie_sentiment_m1.h5',
    ),

    keras.callbacks.ReduceLROnPlateau(
        patience=1,
        factor=0.1,
    )
]

# layer developing
text_input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(max_words, 50)(text_input_layer)
text_layer = Conv1D(256, 3, activation='relu')(embedding_layer)
text_layer = MaxPooling1D(3)(text_layer)
text_layer = Conv1D(256, 3, activation='relu')(text_layer)
text_layer = MaxPooling1D(3)(text_layer)
text_layer = Conv1D(256, 3, activation='relu')(text_layer)
text_layer = MaxPooling1D(3)(text_layer)
text_layer = Conv1D(256, 3, activation='relu')(text_layer)
text_layer = MaxPooling1D(3)(text_layer)
text_layer = Conv1D(256, 3, activation='relu')(text_layer)
text_layer = MaxPooling1D(3)(text_layer)
text_layer = GlobalMaxPooling1D()(text_layer)
text_layer = Dense(256, activation='relu')(text_layer)
output_layer = Dense(1, activation='sigmoid')(text_layer)
model = Model(text_input_layer, output_layer)
model.summary()
model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 500, 50)           500000    
_________________________________________________________________
conv1d (Conv1D)              (None, 498, 256)          38656     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 166, 256)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 164, 256)          196864    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 54, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 52, 256)           196864

In [22]:
# multi-input test
#history = model.fit(x_train, y_train, epochs=50, batch_size=128, callbacks=callback_list, validation_data=(x_val, y_val))
history = model.fit(x_train, y_train, epochs=50, batch_size=128, validation_data=(x_val, y_val))

Epoch 1/50
1/1 [==============================] - 0s 126ms/step - loss: -0.1374 - acc: 0.3333
Epoch 2/50
1/1 [==============================] - 0s 76ms/step - loss: -1.2078 - acc: 0.3333
Epoch 3/50
1/1 [==============================] - 0s 72ms/step - loss: -8.4243 - acc: 0.3333
Epoch 4/50
1/1 [==============================] - 0s 64ms/step - loss: -42.6562 - acc: 0.3333
Epoch 5/50
1/1 [==============================] - 0s 73ms/step - loss: -165.8541 - acc: 0.3333
Epoch 6/50
1/1 [==============================] - 0s 77ms/step - loss: -536.3954 - acc: 0.3333
Epoch 7/50
1/1 [==============================] - 0s 72ms/step - loss: -1493.8833 - acc: 0.3333
Epoch 8/50
1/1 [==============================] - 0s 68ms/step - loss: -3681.5793 - acc: 0.3333
Epoch 9/50
1/1 [==============================] - 0s 71ms/step - loss: -8146.4150 - acc: 0.3333
Epoch 10/50
1/1 [==============================] - 0s 77ms/step - loss: -16578.8027 - acc: 0.3333
Epoch 11/50
1/1 [==============================] -

In [29]:
def review_rating(score, decoded_review):
    if float(score) >= 0.9:
        print('Review: {}\nSentiment: Strongly Positive\nScore: {}'.format(decoded_review, score))
    elif float(score) >= 0.7 and float(score) < 0.9:
        print('Review: {}\nSentiment: Positive\nScore: {}'.format(decoded_review, score))
    elif float(score) >= 0.5 and float(score) < 0.7:
        print('Review: {}\nSentiment: Okay\nScore: {}'.format(decoded_review, score))
    else:
        print('Review: {}\nSentiment: Negative\nScore: {}'.format(decoded_review, score))
    print('\n\n')

In [30]:
def decode_review(text_list):
    word_index = tokenizer.word_index
    sequences = tokenizer.texts_to_sequences(text_list)
    data = pad_sequences(sequences, maxlen=500)

    # decode the words
    reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
    decoded_review = ' '.join([reverse_word_index.get(i, '?') for i in sequences[0]])
    return decoded_review, data

In [31]:
def score_review(source=None, file_type=None):
    '''
    source: the text, as either a string or a list of strings
    file_type: (str): indicating whether we expecting a file containing the
    text data or a directory containing a list files holding the text
    options: 'file' or 'dir'
    '''
    text_list = list()
    if isinstance(source, str) and file_type is None:
        text_list.append(source)
        decoded_review, data = decode_review(text_list)
        # make prediction
        score = model.predict(data)[0][0]
        review_rating(score, decoded_review)
    
    if isinstance(source, list) and file_type is None:
        for item in source:
            text_list = list()
            text_list.append(item)
            decoded_review, data = decode_review(text_list)
            score = model.predict(data)[0][0]
            review_rating(score, decoded_review)
    
    if isinstance(source, str) and file_type == 'file':
        file_data = open(source).read()
        text_list.append(file_data)
        decoded_review, data = decode_review(text_list)
        # make prediction
        score = model.predict(data)[0][0]
        review_rating(score, decoded_review)
    
    if isinstance(source, str) and file_type == 'dir':
        file_content_holder = list()
        for fname in os.listdir(source):
            if fname[-4:] == '.txt':
                f = open(os.path.join(source, fname))
                file_content_holder.append(f.read())
                f.close()
        for item in file_content_holder:
            text_list = list()
            text_list.append(item)
            decoded_review, data = decode_review(text_list)
            score = model.predict(data)[0][0]
            review_rating(score, decoded_review)

In [33]:
score_review('D:/SentimentAnalysis/aclImdb/test/neg', file_type='dir')

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 1757: character maps to <undefined>